In [ ]:
import openrouteservice as ors
import folium


In [86]:
pip install folium

In [87]:
apiKey='5b3ce3597851110001cf624879edeff0b8c54e7e83830c78f5447762'
client = ors.Client(key='5b3ce3597851110001cf624879edeff0b8c54e7e83830c78f5447762')


In [88]:
import operator
from functools import reduce

m = folium.Map(location=list([ 10.997480123197462, 77.01116519467452]), tiles="cartodbpositron", zoom_start=13)

coords = [[10.997480123197462, 77.01116519467452], [11.011062161407962, 76.98302718384372]]
coords=[list(reversed(i)) for i in coords]
route = client.directions(coordinates=coords,
                          profile='foot-walking',
                          format='geojson')

waypoints = list(dict.fromkeys(reduce(operator.concat, list(map(lambda step: step['way_points'], route['features'][0]['properties']['segments'][0]['steps'])))))

folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color="blue").add_to(m)

folium.PolyLine(locations=[list(reversed(route['features'][0]['geometry']['coordinates'][index])) for index in waypoints], color="red").add_to(m)

m

find route for given pair of source and destination locations


In [89]:

def findroute(coords):
  coords=[list(reversed(i)) for i in coords]
  route = client.directions(coordinates=coords,
                          profile='foot-walking',
                          format='geojson')
  return route['features'][0]['geometry']['coordinates']

In [90]:
"""coordinates = list(
  (9.970093, 48.477473),
  (9.207916, 49.153868),
  (37.573242, 55.801281),
  (115.663757,38.106467)
)
"""
"""
coordinates=[[9.970093, 48.477473], [9.207916, 49.153868], [37.573242, 55.801281], [115.663757, 38.106467]]

# query for duration and distance in km
res =ors_matrix(coordinates, metrics = c("duration", "distance"), units = "km")

# duration in hours
#res$durations / 3600
#>            [,1]       [,2]     [,3]      [,4]
#> [1,]   0.000000   1.604628 25.18577 109.65616
#> [2,]   1.573203   0.000000 25.07776 109.54814
#> [3,]  25.131819  24.989258  0.00000  85.57442
#> [4,] 109.457067 109.314506 85.33261   0.00000

# distance in km
#res$distances
#>         [,1]    [,2]    [,3]    [,4]
#> [1,]    0.00  153.50 2410.79 9860.26
#> [2,]  153.81    0.00 2385.20 9834.68
#> [3,] 2358.56 2334.40    0.00 7501.23
#> [4,] 9790.67 9766.52 7449.31    0.00
"""

'\ncoordinates=[[9.970093, 48.477473], [9.207916, 49.153868], [37.573242, 55.801281], [115.663757, 38.106467]]\n\n# query for duration and distance in km\nres =ors_matrix(coordinates, metrics = c("duration", "distance"), units = "km")\n\n# duration in hours\n#res$durations / 3600\n#>            [,1]       [,2]     [,3]      [,4]\n#> [1,]   0.000000   1.604628 25.18577 109.65616\n#> [2,]   1.573203   0.000000 25.07776 109.54814\n#> [3,]  25.131819  24.989258  0.00000  85.57442\n#> [4,] 109.457067 109.314506 85.33261   0.00000\n\n# distance in km\n#res$distances\n#>         [,1]    [,2]    [,3]    [,4]\n#> [1,]    0.00  153.50 2410.79 9860.26\n#> [2,]  153.81    0.00 2385.20 9834.68\n#> [3,] 2358.56 2334.40    0.00 7501.23\n#> [4,] 9790.67 9766.52 7449.31    0.00\n'

load and store latitude and longitude data

In [91]:
import json


with open('input3.json') as f:
    data = json.load(f)


kitchen_coordinates = [coord for coord in data['kitchens'].values()]


customer_coordinates = [coord for coord in data['customers'].values()]
all_coordinates = kitchen_coordinates + customer_coordinates

print("Kitchen Coordinates:", kitchen_coordinates)
print("Customer Coordinates:", customer_coordinates)
print("All Coordinates:", all_coordinates)
orders = data['orders']
max_rider_capacity = data['max_rider_capacity']


Kitchen Coordinates: [[11.055677947474992, 77.0657629297908], [11.080302939124921, 76.94190401520554], [10.95900228800367, 76.9854557122443], [10.99727340737769, 76.99914012640663]]
Customer Coordinates: [[11.064857505996498, 77.09354408383314], [11.059717894092417, 76.94513062197728], [10.924174927712457, 77.03678244820243], [11.003166596428468, 77.04188723253264]]
All Coordinates: [[11.055677947474992, 77.0657629297908], [11.080302939124921, 76.94190401520554], [10.95900228800367, 76.9854557122443], [10.99727340737769, 76.99914012640663], [11.064857505996498, 77.09354408383314], [11.059717894092417, 76.94513062197728], [10.924174927712457, 77.03678244820243], [11.003166596428468, 77.04188723253264]]


In [92]:
def reverse(li):
  x=[list(reversed(x)) for x in li]
  return x

Computing Distance matrix for kitchens,customers and both included


In [93]:
import requests
coords = [list(reversed([10.997480123197462, 77.01116519467452])),list(reversed( [11.011062161407962, 76.98302718384372]))]
y=[[9.70093,48.477473],[9.207916,49.153868],[37.573242,55.801281],[115.663757,38.106467]]

body1 = {"locations":reverse(kitchen_coordinates),"metrics":["distance"],"units":"km"}
body2 = {"locations":reverse(customer_coordinates),"metrics":["distance"],"units":"km"}
body3 = {"locations":reverse(all_coordinates),"metrics":["distance"],"units":"km"}


headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': apiKey,
    'Content-Type': 'application/json; charset=utf-8'
}
kitchen_matrix = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body1, headers=headers)
customer_matrix = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body2, headers=headers)
all_matrix = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body3, headers=headers)

#print(call.status_code, call.reason)
#print(call.text)

In [94]:
import json


data1 = json.loads(kitchen_matrix.text)
kitchenmatrix = data1['distances']
data2 = json.loads(customer_matrix.text)
customermatrix = data2['distances']
data3 = json.loads(all_matrix.text)
allmatrix = data3['distances']

for row in customermatrix:
    print(row)


[0.0, 32.06, 23.81, 14.19]
[22.32, 0.0, 21.37, 17.07]
[20.31, 22.07, 0.0, 14.09]
[11.57, 17.87, 15.13, 0.0]


In [95]:
print(kitchenmatrix)

[[0.0, 20.27, 20.65, 15.33], [20.67, 0.0, 16.76, 14.06], [19.82, 16.42, 0.0, 5.57], [11.53, 14.71, 5.85, 0.0]]


Read json file and store it

In [96]:

kitchens_details = {}
for kitchen_name, kitchen_coord in data['kitchens'].items():
    kitchens_details[kitchen_name] = kitchen_coord


customers_details = {}
for customer_name, customer_coord in data['customers'].items():
    customers_details[customer_name] = customer_coord


print("Kitchen Details:")
for kitchen_name, coord in kitchens_details.items():
    print(f"{kitchen_name}: Coordinates - {coord}")

print("\nCustomer Details:")
for customer_name, coord in customers_details.items():
    print(f"{customer_name}: Coordinates - {coord}")

orders_details = {}
for order_name, order_info in data['orders'].items():
    orders_details[order_name] = {
        'kitchen': order_info['kitchen'],
        'customer': order_info['customer'],
        'expected_ready_time': order_info['expected_ready_time']
    }


max_rider_capacity = data['max_rider_capacity']


print("Orders Details:")
for order_name, details in orders_details.items():
    print(f"Order {order_name}: Kitchen - {details['kitchen']}, Customer - {details['customer']}, Expected Ready Time - {details['expected_ready_time']}")

print(f"\nMaximum Rider Capacity: {max_rider_capacity}")

Kitchen Details:
k1: Coordinates - [11.055677947474992, 77.0657629297908]
k2: Coordinates - [11.080302939124921, 76.94190401520554]
k3: Coordinates - [10.95900228800367, 76.9854557122443]
k4: Coordinates - [10.99727340737769, 76.99914012640663]

Customer Details:
c1: Coordinates - [11.064857505996498, 77.09354408383314]
c2: Coordinates - [11.059717894092417, 76.94513062197728]
c3: Coordinates - [10.924174927712457, 77.03678244820243]
c4: Coordinates - [11.003166596428468, 77.04188723253264]
Orders Details:
Order order1: Kitchen - k1, Customer - c1, Expected Ready Time - 45
Order order2: Kitchen - k1, Customer - c3, Expected Ready Time - 37
Order order3: Kitchen - k3, Customer - c4, Expected Ready Time - 35
Order order4: Kitchen - k2, Customer - c4, Expected Ready Time - 20
Order order5: Kitchen - k4, Customer - c2, Expected Ready Time - 79

Maximum Rider Capacity: 5


In [97]:
orders_details


{'order1': {'kitchen': 'k1', 'customer': 'c1', 'expected_ready_time': 45},
 'order2': {'kitchen': 'k1', 'customer': 'c3', 'expected_ready_time': 37},
 'order3': {'kitchen': 'k3', 'customer': 'c4', 'expected_ready_time': 35},
 'order4': {'kitchen': 'k2', 'customer': 'c4', 'expected_ready_time': 20},
 'order5': {'kitchen': 'k4', 'customer': 'c2', 'expected_ready_time': 79}}

In [98]:
"""
orders_details = {
    'order1': {'kitchen': 'k1', 'customer': 'c1', 'expected_ready_time': 45},
    'order2': {'kitchen': 'k1', 'customer': 'c1', 'expected_ready_time': 37},
    'order3': {'kitchen': 'k3', 'customer': 'c2', 'expected_ready_time': 35},
    'order4': {'kitchen': 'k2', 'customer': 'c1', 'expected_ready_time': 20},
    'order5': {'kitchen': 'k4', 'customer': 'c1', 'expected_ready_time': 79}
}
"""

"\norders_details = {\n    'order1': {'kitchen': 'k1', 'customer': 'c1', 'expected_ready_time': 45},\n    'order2': {'kitchen': 'k1', 'customer': 'c1', 'expected_ready_time': 37},\n    'order3': {'kitchen': 'k3', 'customer': 'c2', 'expected_ready_time': 35},\n    'order4': {'kitchen': 'k2', 'customer': 'c1', 'expected_ready_time': 20},\n    'order5': {'kitchen': 'k4', 'customer': 'c1', 'expected_ready_time': 79}\n}\n"

Assign orders based on rules using a Greedy algorithm

In [99]:
from collections import defaultdict
def assign_orders(orders_details, max_rider_capacity):
    riders = defaultdict(list)
    assigned_orders = set()

    for order, details in orders_details.items():
        kitchen = details['kitchen']
        customer = details['customer']
        ready_time = details['expected_ready_time']
        assigned_to_rider = False


        for rider, orders in riders.items():
            if len(orders) < max_rider_capacity and \
                    all(orders_details[assigned_order]['kitchen'] == kitchen and
                        orders_details[assigned_order]['customer'] == customer and
                        abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10
                        for assigned_order in orders):
                orders.append(order)
                assigned_orders.add(order)
                assigned_to_rider = True
                break


        if not assigned_to_rider:
            for assigned_order in assigned_orders:
                if (orders_details[assigned_order]['customer'] == customer and
                        abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10):
                    route1 = findroute([orders_details[assigned_order]['kitchen'], customer])
                    route2 = findroute([kitchen, customer])
                    if any((coord in route2) for coord in route1):
                        rider = assigned_order.split('_')[0]
                        if len(riders[rider]) < max_rider_capacity:
                            riders[rider].append(order)
                            assigned_orders.add(order)
                            assigned_to_rider = True
                            break


        if not assigned_to_rider:
            for assigned_order in assigned_orders:
                if (orders_details[assigned_order]['kitchen'] == kitchen and
                        abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10):
                    if len(riders[assigned_order]) < max_rider_capacity:
                        riders[assigned_order].append(order)
                        assigned_orders.add(order)
                        assigned_to_rider = True
                        break

        if not assigned_to_rider:
            for assigned_order in assigned_orders:
                if (orders_details[assigned_order]['customer'] == customer and
                        abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10):
                    route1 = findroute([orders_details[assigned_order]['kitchen'], customer])
                    route2 = findroute([kitchen, customer])
                    if any((coord in route2) for coord in route1):
                        rider = assigned_order.split('_')[0]
                        if len(riders[rider]) < max_rider_capacity:
                            riders[rider].append(order)
                            assigned_orders.add(order)
                            assigned_to_rider = True
                            break

        if not assigned_to_rider:
            rider_id = f"rider_{len(riders) + 1}"
            riders[rider_id].append(order)
            assigned_orders.add(order)

        elif not assigned_to_rider:
            for assigned_order in assigned_orders:
                if abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10:
                    route1 = findroute([orders_details[assigned_order]['kitchen'], orders_details[assigned_order]['customer']])
                    route2 = findroute([kitchen, customer])
                    if any((coord in route2) for coord in route1):
                        rider = assigned_order.split('_')[0]
                        if len(riders[rider]) < max_rider_capacity:
                            riders[rider].append(order)
                            assigned_orders.add(order)
                            assigned_to_rider = True
                            break



        elif not assigned_to_rider:
            for assigned_order in assigned_orders:
                if (orders_details[assigned_order]['kitchen'] == kitchen and
                        abs(orders_details[assigned_order]['expected_ready_time'] - ready_time) <= 10):
                    route1 = findroute([orders_details[assigned_order]['customer'], customer])
                    route2 = findroute([customer, orders_details[assigned_order]['customer']])
                    if any((coord in route2) for coord in route1):
                        rider = assigned_order.split('_')[0]
                        if len(riders[rider]) < max_rider_capacity:
                            riders[rider].append(order)
                            assigned_orders.add(order)
                            assigned_to_rider = True
                            break

    return riders

assigned_orders = assign_orders(orders_details, max_rider_capacity)
#for rider, assigned in assigned_orders.items():
 #   print(f"Rider {rider}: Orders {assigned}")
print(assigned_orders)
for i in range(1,10):
  if 'order'+str(i) in assigned_orders:
    assigned_orders['rider_'+str(i)].extend(assigned_orders['order'+str(i)])
    assigned_orders['rider_'+str(i)]=list(set(assigned_orders['rider_'+str(i)]))
    del assigned_orders['order'+str(i)]
for rider, assigned in assigned_orders.items():
    print(f" {rider}: Orders {assigned}")

defaultdict(<class 'list'>, {'rider_1': ['order1'], 'order1': ['order2'], 'rider_3': ['order3'], 'rider_4': ['order4'], 'rider_5': ['order5']})
 rider_1: Orders ['order2', 'order1']
 rider_3: Orders ['order3']
 rider_4: Orders ['order4']
 rider_5: Orders ['order5']


In [100]:
for i in range(1,10):
  if 'order'+str(i) in assigned_orders:
    assigned_orders['rider_'+str(i)].extend(assigned_orders['order'+str(i)])
    assigned_orders['rider_'+str(i)]=list(set(assigned_orders['rider_'+str(i)]))
    del assigned_orders['order'+str(i)]

In [101]:
kitchenmatrix

[[0.0, 20.27, 20.65, 15.33],
 [20.67, 0.0, 16.76, 14.06],
 [19.82, 16.42, 0.0, 5.57],
 [11.53, 14.71, 5.85, 0.0]]

In [102]:
assigned_orders

defaultdict(list,
            {'rider_1': ['order2', 'order1'],
             'rider_3': ['order3'],
             'rider_4': ['order4'],
             'rider_5': ['order5']})

display the routes in map


In [103]:
import operator
from functools import reduce
colors=['blue','cyan','green','black','magenta','red','white','yellow']
i=0
for rider,details in assigned_orders.items():
  for ord in details:
   # m = folium.Map(location=list([ 10.997480123197462, 77.01116519467452]), tiles="cartodbpositron", zoom_start=13)

    k=orders_details[ord]['kitchen']
    c=orders_details[ord]['customer']
    coords=[kitchens_details[k],customers_details[c]]
    route = client.directions(coordinates=reverse(coords),
                          profile='foot-walking',
                          format='geojson')
    waypoints = list(dict.fromkeys(reduce(operator.concat, list(map(lambda step: step['way_points'], route['features'][0]['properties']['segments'][0]['steps'])))))

    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color=colors[i]).add_to(m)

   # folium.PolyLine(locations=[list(reversed(route['features'][0]['geometry']['coordinates'][index])) for index in waypoints], color=colors[i]).add_to(m)
  i+=1
m



In [104]:
kitchens_details


{'k1': [11.055677947474992, 77.0657629297908],
 'k2': [11.080302939124921, 76.94190401520554],
 'k3': [10.95900228800367, 76.9854557122443],
 'k4': [10.99727340737769, 76.99914012640663]}

In [105]:
import operator
from functools import reduce

m = folium.Map(location=list([ 10.997480123197462, 77.01116519467452]), tiles="cartodbpositron", zoom_start=13)

coords = [[10.997480123197462, 77.01116519467452], [11.011062161407962, 76.98302718384372]]
coords=[list(reversed(i)) for i in coords]
route = client.directions(coordinates=coords,
                          profile='foot-walking',
                          format='geojson')

waypoints = list(dict.fromkeys(reduce(operator.concat, list(map(lambda step: step['way_points'], route['features'][0]['properties']['segments'][0]['steps'])))))

folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color="blue").add_to(m)

folium.PolyLine(locations=[list(reversed(route['features'][0]['geometry']['coordinates'][index])) for index in waypoints], color="red").add_to(m)

m